# Logistic Regression Consulting Project

## Binary Customer Churn

A marketing agency has many customers that use their service to produce ads for the client/customer websites. They've noticed that they have quite a bit of churn in clients. They basically randomly assign account managers right now, but want you to create a machine learning model that will help predict which customers will churn (stop buying their service) so that they can correctly assign the customers most at risk to churn an account manager. Luckily they have some historical data, can you help them out? Create a classification algorithm that will help classify whether or not a customer churned. Then the company can test this against incoming data for future customers to predict which customers will churn and assign them an account manager.

The data is saved as customer_churn.csv. Here are the fields and their definitions:

    Name : Name of the latest contact at Company
    Age: Customer Age
    Total_Purchase: Total Ads Purchased
    Account_Manager: Binary 0=No manager, 1= Account manager assigned
    Years: Totaly Years as a customer
    Num_sites: Number of websites that use the service.
    Onboard_date: Date that the name of the latest contact was onboarded
    Location: Client HQ Address
    Company: Name of Client Company
    
Once you've created the model and evaluated it, test out the model on some new data (you can think of this almost like a hold-out set) that your client has provided, saved under new_customers.csv. The client wants to know which customers are most likely to churn given this data (they don't have the label yet).

In [1]:
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder.appName('myLog').getOrCreate()

In [3]:
data = spark.read.csv('customer_churn.csv',inferSchema=True,header=True)

In [4]:
data.printSchema()

root
 |-- Names: string (nullable = true)
 |-- Age: double (nullable = true)
 |-- Total_Purchase: double (nullable = true)
 |-- Account_Manager: integer (nullable = true)
 |-- Years: double (nullable = true)
 |-- Num_Sites: double (nullable = true)
 |-- Onboard_date: string (nullable = true)
 |-- Location: string (nullable = true)
 |-- Company: string (nullable = true)
 |-- Churn: integer (nullable = true)



In [5]:
data.columns

['Names',
 'Age',
 'Total_Purchase',
 'Account_Manager',
 'Years',
 'Num_Sites',
 'Onboard_date',
 'Location',
 'Company',
 'Churn']

In [7]:
data.describe().show()

+-------+-------------+-----------------+-----------------+------------------+-----------------+------------------+-------------------+--------------------+--------------------+-------------------+
|summary|        Names|              Age|   Total_Purchase|   Account_Manager|            Years|         Num_Sites|       Onboard_date|            Location|             Company|              Churn|
+-------+-------------+-----------------+-----------------+------------------+-----------------+------------------+-------------------+--------------------+--------------------+-------------------+
|  count|          900|              900|              900|               900|              900|               900|                900|                 900|                 900|                900|
|   mean|         null|41.81666666666667|10062.82403333334|0.4811111111111111| 5.27315555555555| 8.587777777777777|               null|                null|                null|0.16666666666666666|
| stddev| 

## Transform Data

In [8]:
from pyspark.ml.feature import VectorAssembler

In [9]:
assembler = VectorAssembler(inputCols=['Age',
 'Total_Purchase',
 'Account_Manager',
 'Years',
 'Num_Sites'],outputCol='features')

In [10]:
output = assembler.transform(data)

In [13]:
final_data = output.select('features','churn')

## Build Model

In [15]:
train_churn,test_churn = final_data.randomSplit([0.7,0.3])

In [16]:
from pyspark.ml.classification import LogisticRegression

In [17]:
lr_churn = LogisticRegression(labelCol='churn')

In [20]:
fit_model = lr_churn.fit(train_churn)

In [23]:
train_sum = fit_model.summary

In [25]:
train_sum.predictions.describe().show()

+-------+------------------+-------------------+
|summary|             churn|         prediction|
+-------+------------------+-------------------+
|  count|               625|                625|
|   mean|            0.1728|             0.1328|
| stddev|0.3783771012749461|0.33963026653806616|
|    min|               0.0|                0.0|
|    max|               1.0|                1.0|
+-------+------------------+-------------------+



## Evaluate Results

In [26]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator

In [27]:
my_eval = BinaryClassificationEvaluator(rawPredictionCol='prediction',
                                       labelCol='churn')

In [36]:
result = fit_model.evaluate(test_churn)

In [37]:
result.predictions.show()

+--------------------+-----+--------------------+--------------------+----------+
|            features|churn|       rawPrediction|         probability|prediction|
+--------------------+-----+--------------------+--------------------+----------+
|[28.0,11128.95,1....|    0|[4.25766808386807...|[0.98604230215926...|       0.0|
|[29.0,9378.24,0.0...|    0|[4.80685093523129...|[0.99189270715295...|       0.0|
|[29.0,12711.15,0....|    0|[5.53647810304556...|[0.99607508561264...|       0.0|
|[30.0,8874.83,0.0...|    0|[3.13160724848535...|[0.95817784775792...|       0.0|
|[30.0,10744.14,1....|    1|[1.65007291831513...|[0.83890090529842...|       0.0|
|[30.0,11575.37,1....|    1|[4.06600765179332...|[0.98314331604089...|       0.0|
|[30.0,12788.37,0....|    0|[2.68640976653586...|[0.93621993649753...|       0.0|
|[30.0,13473.35,0....|    0|[2.97674222836026...|[0.95151228949386...|       0.0|
|[31.0,7073.61,0.0...|    0|[2.95811328439120...|[0.95064554710548...|       0.0|
|[32.0,8011.38,0

In [35]:
#AUC
churn_eval = BinaryClassificationEvaluator(rawPredictionCol='prediction',
                                           labelCol='churn')

In [40]:
churn_eval.evaluate(result.predictions)

0.7578172900061312

## New Data

In [41]:
new_customers = spark.read.csv('new_customers.csv',inferSchema=True,
                              header=True)

In [42]:
new_customers.printSchema()

root
 |-- Names: string (nullable = true)
 |-- Age: double (nullable = true)
 |-- Total_Purchase: double (nullable = true)
 |-- Account_Manager: integer (nullable = true)
 |-- Years: double (nullable = true)
 |-- Num_Sites: double (nullable = true)
 |-- Onboard_date: string (nullable = true)
 |-- Location: string (nullable = true)
 |-- Company: string (nullable = true)



In [43]:
test_new_customers = assembler.transform(new_customers)

In [44]:
final_lr_model = lr_churn.fit(final_data)

In [45]:
final_results = final_lr_model.transform(test_new_customers)

In [46]:
final_results.select('Company','prediction').show()

+----------------+----------+
|         Company|prediction|
+----------------+----------+
|        King Ltd|       0.0|
|   Cannon-Benson|       1.0|
|Barron-Robertson|       1.0|
|   Sexton-Golden|       1.0|
|        Wood LLC|       0.0|
|   Parks-Robbins|       1.0|
+----------------+----------+

